# CO Data Aggregation Time Series

Aggregates CO values by administrative boundaries and a regular hexagon grid.

To convert from mol/m<sup>2</sup> to molecules/cm<sup>2</sup>, multiply by 6.022140857e+19 (6.022140857 * 10<sup>19</sup>).

# Environment Setup

In [1]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import packages
import os
import glob
import seaborn as sns
import pandas as pd
import geopandas as gpd
import sentinel as stl

<IPython.core.display.Javascript object>

In [3]:
# Set Options
# sns.set(font_scale=1.5, style="whitegrid")
sns.set(font_scale=1.5)
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
pd.set_option("precision", 15)

<IPython.core.display.Javascript object>

In [4]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

Working directory: C:\PSU\08-covid19-remote-sensing-fusion\00-git-repos\sentinel-5p-no2


<IPython.core.display.Javascript object>

# User-Defined Variables

In [5]:
# Set paths to South hexagon grid and daily rasters
south_korea_hexagon_grid_path = os.path.join(
    "03-processed-data",
    "vector",
    "south-korea",
    "south_korea_hexagon_grid.shp",
)

co_rasters_folder = os.path.join(
    "03-processed-data", "raster", "south-korea", "carbon-monoxide"
)

<IPython.core.display.Javascript object>

# Data Acquisition and Preprocessing

In [6]:
# Get all CO daily rasters into sorted list
co_rasters = sorted(glob.glob(os.path.join(co_rasters_folder, "*.tif")))
co_rasters[:10]

['03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-01-T030235Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-01-T044404Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-02-T042503Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-03-T040600Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-04-T034657Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-05-T032754Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-05-T050924Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-06-T030852Z.tif',
 '03-processed-data\\raster\\south-korea\\carbon-monoxide\\S5P-OFFL-L3-CO-MOL-PER-M2-2018-07-06-T045021Z.tif',
 

<IPython.core.display.Javascript object>

# Data Processing

In [7]:
# Create list of geodataframes aggegated to the hexagon grid
# Takes ~20-30 minutes
aggregated_hexagon_grids = [
    stl.aggregate_raster_data(
        raster_path=raster,
        vector_path=south_korea_hexagon_grid_path,
        zonal_statistics="count mean",
    )
    for raster in co_rasters
]

<IPython.core.display.Javascript object>

In [13]:
# Create list of dataframes with acquisition date for the column name
renamed_data = []
for index, data in enumerate(aggregated_hexagon_grids):
    acquisition_date = os.path.basename(co_rasters[index])[26:45]
    renamed = data.rename(columns={"mean": acquisition_date})
    renamed_data.append(renamed[[acquisition_date]])

<IPython.core.display.Javascript object>

In [15]:
# Get geodataframe with all days of aggregated data (merge geometry with values)
hexagon_grid_gdf = (
    gpd.read_file(south_korea_hexagon_grid_path)
    .drop(columns=["Id"], axis=1)
    .merge(pd.concat(renamed_data, axis=1), left_index=True, right_index=True)
)

<IPython.core.display.Javascript object>

In [16]:
# hexagon_grid_gdf.head()

,GRID_ID,geometry,2018-07-01-T030235Z,2018-07-01-T044404Z,2018-07-02-T042503Z,2018-07-03-T040600Z,2018-07-04-T034657Z,2018-07-05-T032754Z,2018-07-05-T050924Z,2018-07-06-T030852Z,2018-07-06-T045021Z,2018-07-07-T024947Z,2018-07-07-T043118Z,2018-07-08-T041214Z,2018-07-09-T035310Z,2018-07-10-T033408Z,2018-07-11-T045634Z,2018-07-12-T025601Z,2018-07-12-T043730Z,2018-07-13-T041826Z,2018-07-14-T035923Z,2018-07-15-T034018Z,2018-07-16-T032115Z,2018-07-16-T050245Z,2018-07-17-T030211Z,2018-07-17-T044341Z,2018-07-18-T024307Z,2018-07-18-T042435Z,2018-07-19-T040531Z,2018-07-20-T034626Z,2018-07-21-T032722Z,2018-07-21-T050851Z,2018-07-22-T030818Z,2018-07-22-T044947Z,2018-07-23-T043043Z,2018-07-24-T041137Z,2018-07-25-T035233Z,2018-07-26-T033328Z,2018-07-27-T031423Z,2018-07-28-T025518Z,2018-07-28-T043647Z,2018-07-29-T041741Z,2018-07-30-T035837Z,2018-07-31-T033931Z,2018-08-01-T032027Z,2018-08-01-T050156Z,2018-08-02-T030121Z,2018-08-02-T044250Z,2018-08-03-T042345Z,2018-08-04-T040439Z,2018-08-05-T034533Z,2018-08-06-T032627Z,2018-08-06-T050756Z,2018-08-07-T030722Z,2018-08-07-T044851Z,2018-08-08-T024816Z,2018-08-08-T042946Z,2018-08-09-T041040Z,2018-08-10-T035134Z,2018-08-11-T033227Z,2018-08-13-T025415Z,2018-08-13-T043544Z,2018-08-14-T041638Z,2018-08-15-T035731Z,2018-08-16-T033825Z,2018-08-17-T031919Z,2018-08-17-T050048Z,2018-08-18-T030012Z,2018-08-18-T044141Z,2018-08-19-T042235Z,2018-08-20-T040327Z,2018-08-21-T034422Z,2018-08-22-T032514Z,2018-08-23-T030607Z,2018-08-23-T044736Z,2018-08-25-T040924Z,2018-08-26-T035016Z,2018-08-27-T033108Z,2018-08-28-T031202Z,2018-08-28-T045331Z,2018-08-29-T025255Z,2018-08-29-T043424Z,2018-08-30-T041517Z,2018-08-31-T035610Z,2018-09-01-T033702Z,2018-09-02-T031755Z,2018-09-02-T045924Z,2018-09-03-T025847Z,2018-09-04-T042109Z,2018-09-05-T040202Z,2018-09-06-T034253Z,2018-09-07-T032347Z,2018-09-07-T050516Z,2018-09-08-T030439Z,2018-09-08-T044608Z,2018-09-09-T024531Z,2018-09-09-T042700Z,2018-09-10-T040753Z,2018-09-11-T034845Z,2018-09-12-T032937Z,2018-09-13-T031030Z,2018-09-13-T045159Z,2018-09-14-T043251Z,2018-09-15-T041343Z,2018-09-16-T035436Z,2018-09-17-T033527Z,2018-09-18-T031619Z,2018-09-18-T045748Z,2018-09-19-T025711Z,2018-09-19-T043840Z,2018-09-20-T041932Z,2018-09-22-T034116Z,2018-09-23-T032208Z,2018-09-23-T050337Z,2018-09-24-T030300Z,2018-09-24-T044429Z,2018-09-25-T024351Z,2018-09-25-T042521Z,2018-09-26-T040613Z,2018-09-27-T034705Z,2018-09-28-T032755Z,2018-09-30-T024939Z,2018-09-30-T043109Z,2018-10-01-T041201Z,2018-10-02-T035252Z,2018-10-03-T033344Z,2018-10-04-T031435Z,2018-10-04-T045604Z,2018-10-06-T041747Z,2018-10-07-T035839Z,2018-10-08-T033932Z,2018-10-09-T032023Z,2018-10-10-T030115Z,2018-10-10-T044244Z,2018-10-11-T024205Z,2018-10-11-T042335Z,2018-10-12-T040427Z,2018-10-13-T034519Z,2018-10-14-T032609Z,2018-10-15-T030702Z,2018-10-15-T044832Z,2018-10-16-T024753Z,2018-10-16-T042924Z,2018-10-21-T043510Z,2018-10-22-T023433Z,2018-10-22-T041602Z,2018-10-23-T035654Z,2018-10-24-T033745Z,2018-10-25-T031838Z,2018-10-26-T025929Z,2018-10-26-T044059Z,2018-10-27-T024022Z,2018-10-27-T042151Z,2018-10-28-T040243Z,2018-10-29-T034335Z,2018-10-30-T032427Z,2018-10-31-T044648Z,2018-11-01-T024611Z,2018-11-01-T042740Z,2018-11-05-T045239Z,2018-11-06-T025203Z,2018-11-06-T043332Z,2018-11-07-T041425Z,2018-11-08-T035517Z,2018-11-09-T033610Z,2018-11-10-T031703Z,2018-11-10-T045832Z,2018-11-12-T023849Z,2018-11-12-T042018Z,2018-11-13-T040111Z,2018-11-14-T034205Z,2018-11-15-T032258Z,2018-11-16-T030352Z,2018-11-17-T024445Z,2018-11-17-T042614Z,2018-11-18-T040707Z,2018-11-19-T034801Z,2018-11-21-T030950Z,2018-11-22-T025044Z,2018-11-22-T043213Z,2018-11-23-T023138Z,2018-11-23-T041308Z,2018-11-24-T035402Z,2018-11-25-T033456Z,2018-11-26-T031552Z,2018-11-26-T045721Z,2018-11-27-T025646Z,2018-11-27-T043816Z,2018-11-28-T023741Z,2018-11-28-T041910Z,2018-11-29-T040005Z,2018-11-30-T034100Z,2018-12-02-T044420Z,2018-12-03-T024346Z,2018-12-03-T042515Z,2018-12-04-T040611Z,2018-12-05-T034708Z,2018-12-06-T032803Z,2018-12-07-T030859Z,2018-12-07-T045029Z,2018-12-08-T024955Z,20

<IPython.core.display.Javascript object>

# Data Post-Processing

# Data Visualization

# Data Export

In [18]:
# Export hexagon grid
# CSV
# hexagon_grid_gdf.to_csv(
#     path_or_buf=os.path.join(
#         "03-processed-data",
#         "csv",
#         "south-korea",
#         "aggregate-daily-hexagon-grid-jul-2018-aug-2020.csv",
#     ),
#     sep=",",
#     na_rep="nan",
#     header=True,
#     index=False,
# )

# Shapefile
# hexagon_grid_gdf.to_file(
#     filename=os.path.join(
#         "03-processed-data",
#         "vector",
#         "south-korea",
#         "aggregate_daily_hexagon_grid_jul_2018_aug_2020.shp",
#     ),
#     driver="ESRI Shapefile",
#     index=False,
# )

# Geopackage (~5 minutes)
hexagon_grid_gdf.to_file(
    filename=os.path.join(
        "03-processed-data",
        "vector",
        "south-korea",
        "co_aggregate_daily_hexagon_grid_jul_2018_dec_2020.gpkg",
    ),
    driver="GPKG",
    index=False,
)

<IPython.core.display.Javascript object>

In [19]:
# Read exported geopackaged into geodataframe
hexagon_grid_time_series = gpd.read_file(
    os.path.join(
        "03-processed-data",
        "vector",
        "south-korea",
        "co_aggregate_daily_hexagon_grid_jul_2018_dec_2020.gpkg",
    )
).set_index(keys="GRID_ID", drop=True)

<IPython.core.display.Javascript object>

In [20]:
# Check geodataframe
hexagon_grid_time_series.head()

,2018-07-01-T030235Z,2018-07-01-T044404Z,2018-07-02-T042503Z,2018-07-03-T040600Z,2018-07-04-T034657Z,2018-07-05-T032754Z,2018-07-05-T050924Z,2018-07-06-T030852Z,2018-07-06-T045021Z,2018-07-07-T024947Z,2018-07-07-T043118Z,2018-07-08-T041214Z,2018-07-09-T035310Z,2018-07-10-T033408Z,2018-07-11-T045634Z,2018-07-12-T025601Z,2018-07-12-T043730Z,2018-07-13-T041826Z,2018-07-14-T035923Z,2018-07-15-T034018Z,2018-07-16-T032115Z,2018-07-16-T050245Z,2018-07-17-T030211Z,2018-07-17-T044341Z,2018-07-18-T024307Z,2018-07-18-T042435Z,2018-07-19-T040531Z,2018-07-20-T034626Z,2018-07-21-T032722Z,2018-07-21-T050851Z,2018-07-22-T030818Z,2018-07-22-T044947Z,2018-07-23-T043043Z,2018-07-24-T041137Z,2018-07-25-T035233Z,2018-07-26-T033328Z,2018-07-27-T031423Z,2018-07-28-T025518Z,2018-07-28-T043647Z,2018-07-29-T041741Z,2018-07-30-T035837Z,2018-07-31-T033931Z,2018-08-01-T032027Z,2018-08-01-T050156Z,2018-08-02-T030121Z,2018-08-02-T044250Z,2018-08-03-T042345Z,2018-08-04-T040439Z,2018-08-05-T034533Z,2018-08-06-T032627Z,2018-08-06-T050756Z,2018-08-07-T030722Z,2018-08-07-T044851Z,2018-08-08-T024816Z,2018-08-08-T042946Z,2018-08-09-T041040Z,2018-08-10-T035134Z,2018-08-11-T033227Z,2018-08-13-T025415Z,2018-08-13-T043544Z,2018-08-14-T041638Z,2018-08-15-T035731Z,2018-08-16-T033825Z,2018-08-17-T031919Z,2018-08-17-T050048Z,2018-08-18-T030012Z,2018-08-18-T044141Z,2018-08-19-T042235Z,2018-08-20-T040327Z,2018-08-21-T034422Z,2018-08-22-T032514Z,2018-08-23-T030607Z,2018-08-23-T044736Z,2018-08-25-T040924Z,2018-08-26-T035016Z,2018-08-27-T033108Z,2018-08-28-T031202Z,2018-08-28-T045331Z,2018-08-29-T025255Z,2018-08-29-T043424Z,2018-08-30-T041517Z,2018-08-31-T035610Z,2018-09-01-T033702Z,2018-09-02-T031755Z,2018-09-02-T045924Z,2018-09-03-T025847Z,2018-09-04-T042109Z,2018-09-05-T040202Z,2018-09-06-T034253Z,2018-09-07-T032347Z,2018-09-07-T050516Z,2018-09-08-T030439Z,2018-09-08-T044608Z,2018-09-09-T024531Z,2018-09-09-T042700Z,2018-09-10-T040753Z,2018-09-11-T034845Z,2018-09-12-T032937Z,2018-09-13-T031030Z,2018-09-13-T045159Z,2018-09-14-T043251Z,2018-09-15-T041343Z,2018-09-16-T035436Z,2018-09-17-T033527Z,2018-09-18-T031619Z,2018-09-18-T045748Z,2018-09-19-T025711Z,2018-09-19-T043840Z,2018-09-20-T041932Z,2018-09-22-T034116Z,2018-09-23-T032208Z,2018-09-23-T050337Z,2018-09-24-T030300Z,2018-09-24-T044429Z,2018-09-25-T024351Z,2018-09-25-T042521Z,2018-09-26-T040613Z,2018-09-27-T034705Z,2018-09-28-T032755Z,2018-09-30-T024939Z,2018-09-30-T043109Z,2018-10-01-T041201Z,2018-10-02-T035252Z,2018-10-03-T033344Z,2018-10-04-T031435Z,2018-10-04-T045604Z,2018-10-06-T041747Z,2018-10-07-T035839Z,2018-10-08-T033932Z,2018-10-09-T032023Z,2018-10-10-T030115Z,2018-10-10-T044244Z,2018-10-11-T024205Z,2018-10-11-T042335Z,2018-10-12-T040427Z,2018-10-13-T034519Z,2018-10-14-T032609Z,2018-10-15-T030702Z,2018-10-15-T044832Z,2018-10-16-T024753Z,2018-10-16-T042924Z,2018-10-21-T043510Z,2018-10-22-T023433Z,2018-10-22-T041602Z,2018-10-23-T035654Z,2018-10-24-T033745Z,2018-10-25-T031838Z,2018-10-26-T025929Z,2018-10-26-T044059Z,2018-10-27-T024022Z,2018-10-27-T042151Z,2018-10-28-T040243Z,2018-10-29-T034335Z,2018-10-30-T032427Z,2018-10-31-T044648Z,2018-11-01-T024611Z,2018-11-01-T042740Z,2018-11-05-T045239Z,2018-11-06-T025203Z,2018-11-06-T043332Z,2018-11-07-T041425Z,2018-11-08-T035517Z,2018-11-09-T033610Z,2018-11-10-T031703Z,2018-11-10-T045832Z,2018-11-12-T023849Z,2018-11-12-T042018Z,2018-11-13-T040111Z,2018-11-14-T034205Z,2018-11-15-T032258Z,2018-11-16-T030352Z,2018-11-17-T024445Z,2018-11-17-T042614Z,2018-11-18-T040707Z,2018-11-19-T034801Z,2018-11-21-T030950Z,2018-11-22-T025044Z,2018-11-22-T043213Z,2018-11-23-T023138Z,2018-11-23-T041308Z,2018-11-24-T035402Z,2018-11-25-T033456Z,2018-11-26-T031552Z,2018-11-26-T045721Z,2018-11-27-T025646Z,2018-11-27-T043816Z,2018-11-28-T023741Z,2018-11-28-T041910Z,2018-11-29-T040005Z,2018-11-30-T034100Z,2018-12-02-T044420Z,2018-12-03-T024346Z,2018-12-03-T042515Z,2018-12-04-T040611Z,2018-12-05-T034708Z,2018-12-06-T032803Z,2018-12-07-T030859Z,2018-12-07-T045029Z,2018-12-08-T024955Z,2018-12-08-T043124Z

<IPython.core.display.Javascript object>